In [21]:
import pandas as pd
test = pd.read_csv('./TABLE1_5.csv')
test['Eid']

0        2-s2.0-85053164279
1        2-s2.0-85053164279
2        2-s2.0-85054140369
3        2-s2.0-85041527766
4        2-s2.0-85053470598
                ...        
27122    2-s2.0-85015335911
27123    2-s2.0-85014089683
27124    2-s2.0-85013374519
27125    2-s2.0-84995751226
27126    2-s2.0-84995567069
Name: Eid, Length: 27127, dtype: object

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

# -----------------------------------------------------------------------
# 1. Load Data
# -----------------------------------------------------------------------
data = pd.read_csv('./TABLE1_5.csv')
# ข้อมูลต้องมีคอลัมน์: Eid, Title, Subject_Field, Index_Terms, Has_Funding, Funding_Agency

# แทน NaN ใน Index_Terms ด้วย ""
data['Index_Terms'] = data['Index_Terms'].fillna("").astype(str)

# สร้างฟีเจอร์ข้อความรวม Title + Index_Terms
data['text_all'] = data['Title'].fillna("") + " " + data['Index_Terms'].fillna("")

# -----------------------------------------------------------------------
# 2. แบ่งชุดข้อมูลสำหรับ Has_Funding (Binary Classification)
# -----------------------------------------------------------------------
X = data[['text_all', 'Subject_Field']]
y = data['Has_Funding'].astype(int)  # 0 หรือ 1

X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing สำหรับ Binary Classification
text_feature = 'text_all'
cat_features = ['Subject_Field']

text_transformer = TfidfVectorizer(stop_words='english', max_features=1000)
cat_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor_bin = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_feature),
        ('cat', cat_transformer, cat_features)
    ],
    remainder='drop'
)

model_bin = Pipeline(steps=[
    ('preprocess', preprocessor_bin),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

# Train Binary Model
model_bin.fit(X_train_bin, y_train_bin)

# Evaluate Binary Model
y_pred_bin = model_bin.predict(X_test_bin)
print("Binary Classification (Has_Funding) Report:")
print(classification_report(y_test_bin, y_pred_bin))




Binary Classification (Has_Funding) Report:
              precision    recall  f1-score   support

           0       0.68      0.67      0.67      2728
           1       0.67      0.68      0.67      2698

    accuracy                           0.67      5426
   macro avg       0.67      0.67      0.67      5426
weighted avg       0.67      0.67      0.67      5426

